In [39]:
# Import the required libraries
import hvplot.pandas
import pandas as pd

In [40]:
# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [41]:
# Store the CSV you saved in part one into a DataFrame
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Upernavik,GL,2023-02-27 16:40:14,72.7868,-56.1549,-3.42,89,100,18.81
1,1,Reshetylivka,UA,2023-02-27 16:40:15,49.5650,34.0782,32.79,98,100,10.40
2,2,Krasnoselkup,RU,2023-02-27 16:40:16,65.7000,82.4667,-2.54,98,58,12.15
3,3,Nilanga,IN,2023-02-27 16:40:16,18.1000,76.7667,80.73,21,0,5.10
4,4,Appleton,US,2023-02-27 16:39:41,44.2619,-88.4154,34.11,93,100,11.50


In [42]:
# Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [43]:
# Configure the map plot_1

map_plot_1 = city_data_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM")

# Display the map plot_1
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

In [44]:
# Cofigure the map plot_2
map_plot_2 = city_data_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", frame_width=700, frame_height=500)
# Display the map plot_2
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

In [45]:
# Configure the map plot_3
map_plot_3 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 0.5,
    color = "City"
)

# Display the map plot
map_plot_3

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)

In [46]:
# Configure the map plot_4
map_plot_4 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 0.5,
    color = "City"
)

# Display the map plot
map_plot_4

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)

In [47]:
# Configure the map plot_5
map_plot_5 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "CartoDark",
    frame_width = 700,
    frame_height = 500,
    size = "Wind Speed",
    scale = 0.75,
    color = "City"
)

# Display the map plot
map_plot_5

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Wind Speed)

In [48]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [49]:
preferred_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)
]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Nilanga,IN,2023-02-27 16:40:16,18.1000,76.7667,80.73,21,0,5.10
7,7,Saint-Philippe,RE,2023-02-27 16:40:20,-21.3585,55.7679,80.19,81,47,8.46
9,9,Nokaneng,BW,2023-02-27 16:40:22,-19.6667,22.2667,85.39,30,25,9.71
10,10,Grand Gaube,MU,2023-02-27 16:40:23,-20.0064,57.6608,81.00,76,9,1.99
12,12,Mitsamiouli,KM,2023-02-27 16:40:25,-11.3847,43.2844,81.64,80,89,9.53
16,16,Kavaratti,IN,2023-02-27 16:40:28,10.5669,72.6420,81.37,58,0,2.37
20,20,Cidreira,BR,2023-02-27 16:40:32,-30.1811,-50.2056,78.87,69,0,8.88
22,22,Souillac,MU,2023-02-27 16:40:16,-20.5167,57.5167,80.98,78,40,1.14
24,24,Pedernales,DO,2023-02-27 16:40:34,18.0384,-71.7440,80.49,56,0,7.74
26,26,Saldanha,ZA,2023-02-27 16:40:36,-33.0117,17.9442,80.65,83,10,26.42


In [50]:
preferred_cities_df.count()

City_ID       200
City          200
Country       199
Date          200
Lat           200
Lng           200
Max Temp      200
Humidity      200
Cloudiness    200
Wind Speed    200
dtype: int64

In [59]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Nilanga,IN,80.73,18.1000,76.7667,
7,Saint-Philippe,RE,80.19,-21.3585,55.7679,
9,Nokaneng,BW,85.39,-19.6667,22.2667,
10,Grand Gaube,MU,81.00,-20.0064,57.6608,
12,Mitsamiouli,KM,81.64,-11.3847,43.2844,
16,Kavaratti,IN,81.37,10.5669,72.6420,
20,Cidreira,BR,78.87,-30.1811,-50.2056,
22,Souillac,MU,80.98,-20.5167,57.5167,
24,Pedernales,DO,80.49,18.0384,-71.7440,
26,Saldanha,ZA,80.65,-33.0117,17.9442,


In [60]:
# Import the required libraries
import requests

# Import API key
from config import geoapify_key

In [63]:
# Set initial parameters to search for a hotel.
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
}

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"

    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

 # Get the first hotel from the results and store the name
try:
    hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
except (KeyError, IndexError):
    # If no hotel is found, set the hotel name as "No hotel found".
    hotel_df.loc[index, "Hotel Name"] = "No hotel found"   


In [70]:
# Configure the map
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)